In [1]:
# 알파벳 순서에서 인덱스를 원-핫 인코딩으로 취한다.
import tensorflow as tf
import numpy as np

char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
           'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z']
num_dic = {n:i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [2]:
# 학습용 단어
seq_data = ['word', 'wood', 'deep', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [11]:
# 학습에 사용할 수 있는 형식으로 변환해주는 유틸리티 함수 작성
# 입력값으로 단어의 세글자 알파벳 인덱스를 구한 배열제작
# 출력용으로 마지막 글자의 알파벳 인덱스 
# 입력값 원-핫 인코딩 변환
# 실측값 인코딩 않고 그대로 사용, 손실 ㅎ마수로 다른 것 사용, 자동 변환
def make_batch(seq_data):
    input_batch = []
    target_batch = []
    for seq in seq_data:
        input = [num_dic[n] for n in seq[:-1]]
        target = num_dic[seq[-1]]
        input_batch.append(np.eye(dic_len)[input])
        target_batch.append(target)
        
    return input_batch, target_batch

In [4]:
# 신경망 구현
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len

In [5]:
# 첫 3글자만 단계적 학습하므로 n_step은 3
# 본격적 신경망 모델 구성
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

# 원-핫 아닌 인덱스 그대로 사용, 값 하나뿐인 1차원 배열 입력
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

In [6]:
# 두개 RNN 셀 생성, 심층 신경망 제작, DropoutWrapper로 과적합 방지
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

W0729 18:04:23.496326  8980 deprecation.py:323] From <ipython-input-6-ec2aadbc5025>:2: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [7]:
# 함수 조합하여 심층 순환 신경 망 제작
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

outputs, status = tf.nn.dynamic_rnn(multi_cell, X, dtype = tf.float32)

W0729 18:04:23.999767  8980 deprecation.py:323] From <ipython-input-7-c4901e494a5a>:2: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0729 18:04:24.001044  8980 deprecation.py:323] From <ipython-input-7-c4901e494a5a>:4: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0729 18:04:24.328240  8980 deprecation.py:506] From c:\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the construc

In [8]:
# 출력층 제작
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [9]:
# 손실함수와 최적화로 구성 마무리
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model,labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [13]:
# 신경망 학습 코드, make_batch 이용, seq_data에 저장한 입력, 실츨 분리 최적화 실해
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X:input_batch, Y: target_batch})
    
    print('Epoch:', '%04d'%(epoch +1), 'cost = ', '{:.6f}'.format(loss))
    
print('최적화 완료!')

Epoch: 0001 cost =  3.778096
Epoch: 0002 cost =  2.775152
Epoch: 0003 cost =  1.340362
Epoch: 0004 cost =  1.470062
Epoch: 0005 cost =  0.732277
Epoch: 0006 cost =  1.012854
Epoch: 0007 cost =  0.897003
Epoch: 0008 cost =  0.402283
Epoch: 0009 cost =  0.391643
Epoch: 0010 cost =  0.685368
Epoch: 0011 cost =  0.527102
Epoch: 0012 cost =  0.183882
Epoch: 0013 cost =  0.302258
Epoch: 0014 cost =  0.280659
Epoch: 0015 cost =  0.173113
Epoch: 0016 cost =  0.103174
Epoch: 0017 cost =  0.169920
Epoch: 0018 cost =  0.134919
Epoch: 0019 cost =  0.059107
Epoch: 0020 cost =  0.049404
Epoch: 0021 cost =  0.049807
Epoch: 0022 cost =  0.048226
Epoch: 0023 cost =  0.038015
Epoch: 0024 cost =  0.062969
Epoch: 0025 cost =  0.059888
Epoch: 0026 cost =  0.031552
Epoch: 0027 cost =  0.026134
Epoch: 0028 cost =  0.004014
Epoch: 0029 cost =  0.012228
Epoch: 0030 cost =  0.003386
최적화 완료!


In [15]:
# 예측 단어를 정확도와 함께 출력 하도록 만들자
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

In [16]:
# 예측 모델 돌려보자
input_batch, target_batch = make_batch(seq_data)
predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X:input_batch, Y:target_batch})

In [17]:
predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)
    
print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0
